In [10]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.model_selection import train_test_split
from PIL import Image
from pathlib import Path
import torchvision
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

class SpectrogramDataset(Dataset):
    def __init__(self, spectro_folder):
        self.spectrogram_folders = list(spectro_folder.glob('spectrogram_*'))
        self.transform = transforms.Compose([
            transforms.Resize((512, 512)),
            transforms.ToTensor(),
            transforms.Lambda(lambda x: torch.flatten(x)),
        ])

    def __len__(self):
        return len(self.spectrogram_folders)

    def __getitem__(self, idx):
        folder_path = self.spectrogram_folders[idx]

        # Load the spectrogram image
        img = Image.open(folder_path / 'spectrogram.png').convert('RGB')
        img = self.transform(img)

        # Load the label
        label_path = folder_path / 'label.txt'
        with open(label_path, 'r') as label_file:
            label = int(label_file.read().strip())

        return img, label

# Path to the folder containing your spectrograms
spectro_folder = Path('/Users/madssverker/Documents/GitHub/Bachelor_Thesis/spectrograms')

# Create a custom dataset
Data = SpectrogramDataset(spectro_folder)

batch_size = 100
dataloader = DataLoader(Data, batch_size = batch_size)
# images = np.asarray(images)
# labels = np.asarray(labels)

# Split the dataset into train and test sets
# x_train, y_train, x_test, y_test = train_test_split(Data, test_size=0.2, shuffle=True, stratify=labels)
# print(images)
# # Create data loaders for train and test sets

# train_data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# test_data_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [11]:
for batch in dataloader:
    images, labels = batch
    images = np.asarray(images)
    labels = np.asarray(labels)
    x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, shuffle = True, stratify=labels)

    
    classifier = SVC()

    parameters = [{'gamma': [0.01, 0.001, 0.0001], 'C': [1, 5, 50, 500]}]

    grid_search = GridSearchCV(classifier, parameters)

    grid_search.fit(x_train, y_train)

    best_estimator = grid_search.best_estimator_

    y_pred = best_estimator.predict(x_test)

    score = accuracy_score(y_pred, y_test)

    print('{}% of samples were correctly classfied'.format(str(score * 100)))
    

90.0% of samples were correctly classfied


In [13]:
cm = confusion_matrix(y_test, y_pred)
print(y_test, y_pred)
print(cm)


[1 3 2 1 2 4 5 1 2 4 1 5 3 4 3 2 5 5 3 4] [2 3 2 2 2 4 5 1 2 4 1 5 3 4 3 2 5 5 3 4]
[[2 2 0 0 0]
 [0 4 0 0 0]
 [0 0 4 0 0]
 [0 0 0 4 0]
 [0 0 0 0 4]]
